In [16]:
import pandas as pd
import os
import re
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append(os.path.expanduser("~") + "/collproc/")
from raw import *
from coll import *

In [2]:
df = pd.read_csv("genome_roughness_768_250_501_21_divide.csv")

In [3]:
df['fbase'] = [os.path.basename(item)[:-4] for item in df.localpath]

In [4]:
df['samploc'] = [item.split("/")[-2] for item in df.localpath]

In [5]:
df['catalog'] = df.fbase.apply(extract_catalog_number)

In [6]:
df['manbran'] = df.fbase.apply(extract_two_letter_code)

In [7]:
df['mtrial'] = df.fbase.apply(extract_single_digit)

#### Correct manbrans that were skipped because only one letter

In [8]:
df.manbran[df.manbran.isnull()] = df.fbase[df.manbran.isnull()].apply(lambda x: x.split("_")[1])

#### Correct mtrials

In [9]:
df.mtrial[df.fbase=='2172_kb_'] = '2'

In [10]:
df.mtrial[df.fbase=='97_kv_166'] = '0'
df.mtrial[df.fbase=='97_kv_167'] = '1'

In [11]:
df.mtrial[df.fbase=='2982_kp_'] = '2'

# Enumeration of Cases

**<font color='red'>NOTE: I have not processed the new texture captures that Joitree took. I'll have to do that.</font>**

In [13]:
badcats = []
for catalog in df.catalog.unique():
    tmp = df[df.catalog==catalog]
    tmp.mtrial = tmp.mtrial.apply(int)
    tmp = tmp.sort_values('mtrial')
    
    # if all measurements are from same dir, have same manbran, and trial indices are consecutive, then ignore
    if all([len(tmp.samploc.unique())==1,len(tmp.manbran.unique())==1,is_consecutive(tmp.mtrial)]):
        continue
    else:
        print(catalog,end='\t')
        badcats.append(catalog)

5149	5150	5151	5154	97	1963	82	4213	5198	2198	4231	4945	4949	4783s	1882c	289c	289d	289e	289f	2073z	3202a	

In [14]:
df = df[~df.catalog.isin(badcats)]

In [18]:
pkl(df,'x.pkl')